In [3]:
!pip install overpy

You are using pip version 7.1.2, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [44]:
import overpy

In [45]:
api = overpy.Overpass()

In [6]:
query = """way["name"="Käsröthe"];out;"""

In [46]:
query = """way(47081339);out;"""

In [61]:
query = """(way(161468888);way(47081339););
(._;>;);
out;"""

In [16]:
query = """(node
    (43.7634,11.2153,43.7908,11.289);
    way(bn)->.a
);
(way.a['highway'~'primary|secondary|residential|tertiary|'];>);
out;"""

In [66]:
query = """way["highway"~"primary|secondary|tertiary|residential"](50.394184,10.974431,50.544216,11.24176);
(._;>;);
out;"""

In [67]:
result = api.query(query)

In [68]:
nodes = []

for way in result.ways:
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    
    nodes_ = []
    for node in way.get_nodes(resolve_missing=True):
    	nodes_.append((node.lat, node.lon,))
        
    nodes.append(nodes_)

# print(nodes)

Name: Masserberger Straße
  Highway: secondary
Name: n/a
  Highway: tertiary
Name: Ortsstraße
  Highway: tertiary
Name: n/a
  Highway: secondary
Name: Bärenbachstraße
  Highway: secondary
Name: n/a
  Highway: secondary
Name: Lichtner Straße
  Highway: secondary
Name: n/a
  Highway: tertiary
Name: Jagdshofer Straße
  Highway: tertiary
Name: n/a
  Highway: tertiary
Name: Lehnersgasse
  Highway: tertiary
Name: Marktplatz
  Highway: residential
Name: Bahnhofstraße
  Highway: residential
Name: Blechhammer
  Highway: residential
Name: Rotseifenberg
  Highway: residential
Name: Freiherr-vom-Stein-Straße
  Highway: secondary
Name: Steinacher Straße
  Highway: secondary
Name: n/a
  Highway: secondary
Name: Hämmerer Straße
  Highway: secondary
Name: Poststraße
  Highway: residential
Name: Prachaticer Straße
  Highway: residential
Name: Prachaticer Straße
  Highway: residential
Name: Fichtacher Straße
  Highway: residential
Name: Am Rothenberg
  Highway: residential
Name: n/a
  Highway: secondary

In [71]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

In [52]:
def approximate_parabole(p1, p2, p3):
    (x1,y1) = p1
    (x2,y2) = p2
    (x3,y3) = p3
    c=(y1-y2)/(y2-y3)
    x0=(-x1**2+x2**2+c*(x2**2-x3**2))/(2*(-x1+x2+c*x2-c*x3))
    a=(y1-y2)/((x1-x0)**2-(x2-x0)**2)
    y0=y1-a*(x1-x0)**2
    return 1/a
    #return (a, x0, y0)

In [55]:
import numpy as np

def calculate_angle(p0, p1, p2):
    """
    Calculates angle between (p0,p1) and (p0,p2)
    """
    a = np.array((p0[0]-p1[0],p0[1]-p1[1]))
    b = np.array((p0[0]-p2[0],p0[1]-p2[1]))
    return np.arccos(np.dot(a, b)/np.linalg.norm(a)/np.linalg.norm(b))

In [105]:
def convert_to_meters(p0, p1):
    def to_radians(angle):
        from math import pi
        return angle*pi/180.0

    from math import sin, cos, pi, sqrt, atan2
    lat1, lon1 = p0 
    lat2, lon2 = p1
    R = 6371 # km
    dLat = to_radians(lat2-lat1)
    dLon = to_radians(lon2-lon1)
    lat1 = to_radians(lat1)
    lat2 = to_radians(lat2)

    a = sin(dLat/2) * sin(dLat/2) + sin(dLon/2) * sin(dLon/2) * cos(lat1) * cos(lat2)
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c
    return d

In [92]:
def convert_to_meters(p0, p1):
    """
    http://stackoverflow.com/a/11172685/683408
    """
    lat1, lon1 = p0 
    lat2, lon2 = p1
    from math import sin, cos, pi, sqrt, atan2
    R = 6378.137; # Radius of earth in KM
    dLat = (lat2 - lat1) * pi / 180;
    dLon = (lon2 - lon1) * pi / 180;
    a = sin(dLat/2) * sin(dLat/2) + cos(lat1 * pi / 180) * cos(lat2 * pi / 180) * sin(dLon/2) * sin(dLon/2);
    c = 2 * atan2(sqrt(a), sqrt(1-a));
    d = R * c;
    return d * 1000; # meters

In [106]:
road_classes = ['blue', 'green', 'red', 'black']
points = []
for path in nodes:
    intensity = [0]*len(path)
    total_spass = 0.0
    total_distance = 0.0
    for index in range(len(path)-2):
        p0 = np.array(path[index],dtype=np.float)
        p1 = np.array(path[index+1],dtype=np.float)
        p2 = np.array(path[index+2],dtype=np.float)
        intens = np.linalg.norm(p0-p2)/(np.linalg.norm(p0-p1) + np.linalg.norm(p2-p1))
        #intens = approximate_parabole(p0,p1,p2)
        intens = calculate_angle(p0,p1,p2)
        intensity[index+1] = intens
        points.append([p1, intens])
        total_spass += intens
        # print(convert_to_meters(p0, p1))
        total_distance += convert_to_meters(p0, p1)
    
    if total_distance < 0.01:
        continue
        
    #print('total ' + str(total_spass/total_distance))
    total_spass /= total_distance
    total_spass *= 2000.0
    if total_spass > 10:
        c = 'black'
    elif total_spass > 6:
        c = 'red'
    elif total_spass > 3:
        c = 'green'
    elif total_spass > 1:
        c = 'blue'
    else:
        c = 'gray'
    plt.plot([n[0] for n in path], [n[1] for n in path], linewidth=1, c=c)
    plt.annotate(str(str(round(total_spass,2))), xy=path[len(path)//2])

min_intensity = min(p[1] for p in points)
#for p in points:
#    p[1] = (p[1]-min_intensity)/(1-min_intensity)
bounds = np.array([-0.25, 0, 0.1, 0.2,0.3, 1])
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
# plt.scatter([p[0][0]for p in points], [p[0][1]for p in points], c=[p[1]for p in points],s=40, norm=norm, cmap='RdBu_r')

#for ((x,y), v) in points:
#    plt.annotate(str(round(v,2)), xy=(x,y))
plt.show()